# This notebook must be run with the scvi conda environment within the scvi.sif singularity container

In [ ]:
import scanpy as sc
import loompy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import anndata

In [ ]:
combined_adata = sc.read_h5ad('full_dataset_SCANVI.h5ad')

In [ ]:
# I wanted to orient all of the brains to be in roughly the same position
def x_mirror_matrix():
    return np.array([[-1, 0],
                     [0, 1]])

def y_mirror_matrix():
    return np.array([[1, 0],
                     [0, -1]])

def rotation_matrix(angle):
    theta = np.radians(angle)
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],
                     [s, c]])

# Apply transformations based on batchID
def apply_transformations_inplace(adata, batchID, transformations):
    spatial_data = adata.obsm['spatial']
    batch_indices = adata.obs['batchID'] == batchID

    for idx in np.where(batch_indices)[0]:
        for transformation in transformations:
            spatial_data[idx] = np.dot(transformation, spatial_data[idx])

transformations = {
    '24-mo-female-1': [rotation_matrix(180)],
    '24-mo-female-3': [x_mirror_matrix()],
    '24-mo-female-5': [rotation_matrix(90)],
    '24-mo-male-1': [rotation_matrix(180)],
    '24-mo-male-2': [x_mirror_matrix()],
    '24-mo-male-4': [rotation_matrix(60)],
    '3-mo-female-1': [y_mirror_matrix()],
    '3-mo-female-2': [rotation_matrix(270)],
    '3-mo-female-3': [rotation_matrix(-80)],
    '3-mo-male-1': [x_mirror_matrix()],
    '3-mo-male-2': [rotation_matrix(270)],
    '3-mo-male-3': [rotation_matrix(270)],
    '3-mo-female-1-rev2': [rotation_matrix(-55),y_mirror_matrix()],
    '3-mo-male-3-rev2': [rotation_matrix(-25),y_mirror_matrix()],
    '24-mo-male-4-rev2': [rotation_matrix(90)],
}

for batchID, transform_matrix in transformations.items():
    apply_transformations_inplace(combined_adata, batchID, transform_matrix)
    print(f"Applied transformation to batchID {batchID}")

In [ ]:
combined_adata.obs.loc[combined_adata.obs['leiden'] == '24', 'celltype'] = 'Microglia'
microglia = combined_adata[combined_adata.obs.celltype == 'Microglia']

# save this for parquet creation
microglia.write('baysor_microglia.h5ad')

In [ ]:
#### Okay now this is where we are going to create the loop to do all of the brains ####

# Define the target batchID
target_batchID = '3-mo-female-2'

# Get the spatial coordinates of the target batchID
target_geom = combined_adata[combined_adata.obs['batchID'] == target_batchID]
xJ = target_geom.obsm['spatial'][:, 0]
yJ = target_geom.obsm['spatial'][:, 1]

target_geom.obs['aligned_x'] = np.array(xJ)
target_geom.obs['aligned_y'] = np.array(yJ)


# manually establish 
theta_mappings = {
    '24-mo-female-1': 155,
    '24-mo-female-3': 230,
    '24-mo-female-5': 10,
    '24-mo-male-1': 180,
    '24-mo-male-2': 90,
    '24-mo-male-4': -60,
    '3-mo-female-1': 230,
    '3-mo-female-3': 0,
    '3-mo-male-1': 0,
    '3-mo-male-2': -20,
    '3-mo-male-3': -55,
    '3-mo-male-3-rev2': 0,
    '3-mo-female-1-rev2': 0,
    '24-mo-male-4-rev2':0
}

# generate a list of adata objects:
ad_list = [target_geom]

for batchID in combined_adata.obs['batchID'].unique():
    if batchID != target_batchID:
        # Filter for the current batchID
        source = combined_adata[combined_adata.obs['batchID'] == batchID]
        xI = source.obsm['spatial'][:, 0]
        yI = source.obsm['spatial'][:, 1]

        # Get theta_deg for current batchID from mappings
        theta_deg = theta_mappings.get(batchID, 0)
        theta0 = (np.pi / 180) * -theta_deg

        # Rotation matrix
        L = np.array([[np.cos(theta0), -np.sin(theta0)],
                      [np.sin(theta0), np.cos(theta0)]])

        source_L = np.matmul(L, np.array([xI, yI]))
        xI_L = source_L[0]
        yI_L = source_L[1]

        # Translation matrix
        T = np.array([np.mean(xI) - np.cos(theta0) * np.mean(xI) + np.sin(theta0) * np.mean(yI) - (np.mean(xI) - np.mean(xJ)),
                      np.mean(yI) - np.sin(theta0) * np.mean(xI) - np.cos(theta0) * np.mean(yI) - (np.mean(yI) - np.mean(yJ))])

        xI_L_T = xI_L + T[0]
        yI_L_T = yI_L + T[1]

        # Update the 'aligned_x' and 'aligned_y' columns in old_1.obs
        source.obs['aligned_x'] = xI_L_T
        source.obs['aligned_y'] = yI_L_T

        # Optionally, you can save the updated batch back to ad_viz if needed
        ad_list.append(source)
        
        
#concatenated_ad = anndata.concat(ad_list, index_unique=None)
concatenated_ad_sc = sc.AnnData.concatenate(*ad_list, index_unique=None)

concatenated_ad_sc.obsm['spatial'] = concatenated_ad_sc.obs[["aligned_x", "aligned_y"]].values

concatenated_ad_sc.write('spatial_baysor_rotated.h5ad')